In [54]:
import json
import os
import base64
import requests
import pandas as pd

# 文件夹路径
file_path = 'AlternativeInvestments.json'
folder_path = 'AlternativeInvestments_images'



In [55]:
# Load the data from the file
with open(file_path, "r") as f:
    data = json.load(f)


# 获取文件夹下所有文件的文件名
files = os.listdir(folder_path)

# 筛选文件名中包含 "u" 的文件
filtered_files_u = [file for file in files if 'u' in file]

# 筛选文件名中包含 "dn" 的文件
filtered_files_dn = [file for file in files if 'dn' in file]

# 筛选文件名中包含 "share" 的文件
filtered_files_share = [file for file in files if 'share' in file]

# 输出筛选结果
print(filtered_files_u)

print(filtered_files_dn)

print(filtered_files_share)

[]
['17dn.png', '18dn.png', '19dn.png', '6dn.png']
['share1-6_1.png', 'share1-6_2.png', 'share14-25_1.png', 'share14-25_2.png', 'share14-25_3.png', 'share26-33_1.png', 'share26-33_2.png', 'share26-33_3.png', 'share34-40_1.png', 'share41-46_1.png', 'share47-52_1.png', 'share47-52_2.png', 'share7-13_1.png', 'share7-13_2.png']


In [56]:
for file in filtered_files_u:
    # 提取文件名中的数字部分
    file_number = ''.join(filter(str.isdigit, file))
    
    # 构建完整的图片路径
    image_path = f"{folder_path}/{file}"
    
    # 检查 JSON 数据结构中是否有对应的 question_number
    for entry in data:
        if entry['question_number'] == file_number:
            # 更新对应的 'image' 字段
            entry['image'] = image_path


In [57]:
import re

for file in filtered_files_share:
    # 使用正则表达式从文件名中提取X和Y
    match = re.match(r'share(\d+)-(\d+)_\d+\.png', file)
    if match:
        x = int(match.group(1))
        y = int(match.group(2))
        image_path = f"{folder_path}/{file}"

        # 对于从X到Y（包括Y）的所有question_number
        for q_num in range(x, y + 1):
            # 在data列表中找到匹配的question_number
            for entry in data:
                if entry['question_number'] == str(q_num):
                    # 如果'share_image'字段不存在，创建它并插入到指定位置
                    if 'share_image' not in entry:
                        # 获取字段的顺序
                        keys = list(entry.keys())
                        # 找到'image'字段的位置
                        image_index = keys.index('image')
                        # 在'image'之后插入'share_image'
                        keys.insert(image_index + 1, 'share_image')
                        # 创建新的有序字典以保持字段顺序
                        new_entry = {}
                        for key in keys:
                            if key == 'share_image':
                                new_entry[key] = [image_path]
                            else:
                                new_entry[key] = entry[key]
                        # 更新原始条目
                        entry.clear()
                        entry.update(new_entry)
                    else:
                        # 如果'share_image'字段已经存在，添加新的图片路径
                        if image_path not in entry['share_image']:
                            entry['share_image'].append(image_path)
    else:
        print(f"文件名 {file} 不符合预期的格式。")


In [58]:
data

[{'share_context': ['Tim Wang is a financial adviser specializing in commercial real estate investing. He is\nmeeting with Mark Caudill, a new client who is looking to diversify his investment portfolio by\nadding real estate investments. Caudill has heard about various investment vehicles related to\nreal estate from his friends and is seeking a more in-depth understanding of these investments\nfrom Wang.\nWang begins the meeting by advising Caudill of the many options that are available when\ninvesting in real estate, including the following:\nOption 1 Direct ownership in real estateOption 2 Publicly traded real estate investment\ntrustsOption 3 Publicly traded real estate operating companiesOption 4 Publicly-traded\nresidential mortgage-backed securitiesWang next asks Caudill about his investment preferences.\nCaudill responds by telling Wang that he prefers to invest in equity securities that are highly\nliquid, provide high income, and are not subject to double taxation.\nCaudill 

In [59]:
# 初始化一个字典来存储每个文件的base64编码
base64_images = {}

# 转换每个文件到base64
for file_name in filtered_files_dn:
    file_path = os.path.join(folder_path, file_name)
    print(f"Processing {file_path}")  # 打印正在处理的文件路径

    try:
        # 打开图片文件
        with open(file_path, "rb") as image_file:
            # 转换图片为base64编码
            base64_string = base64.b64encode(image_file.read()).decode('utf-8')
            # 存储编码到字典
            base64_images[file_name] = base64_string
            print(f"Successfully encoded {file_name}")  # 打印成功编码的文件名
    except Exception as e:
        print(f"Error converting {file_name}: {e}")  # 打印出错的文件名和错误消息


Processing AlternativeInvestments_images\17dn.png
Successfully encoded 17dn.png
Processing AlternativeInvestments_images\18dn.png
Successfully encoded 18dn.png
Processing AlternativeInvestments_images\19dn.png
Successfully encoded 19dn.png
Processing AlternativeInvestments_images\6dn.png
Successfully encoded 6dn.png


In [60]:
# OpenAI API Key
api_key = "sk-VUQOgoxNjtiPxFDo895535A3635847B7A903688099089385"

# 存储结果
results = {}

# 创建请求头
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# 处理每张图片
for image_file, base64_string in base64_images.items():
    # 构建 payload，使用 base64 编码的图片，并要求解析其内容
    payload = {
        "model": "gpt-4o",
        "messages": [
            {
                "role": "user",
                "content": [
                    # 提供任务说明，要求解析图片内容
                    {
                        "type": "text",
                        "text": """You will get an image with text and mathematical formulas. Your task is to extract content from images and convert it into a document format. The requirements are as follows:

2. Use inline LaTeX formatting (i.e., $ ... $) for mathematical formulas or equations that are part of a sentence or text. For example, 'The fuction is :RCA = \frac{(UL_A^2 + p \times UL_A \times UL_B)}{UL_p}' should be converted to: 'The fuction is :$RCA = \frac{(UL_A^2 + p \times UL_A \times UL_B)}{UL_p}$.'
1. For any math expressions, convert them to LaTeX code.
3. Use block-level LaTeX format (i.e., $$ ... $$) for stand-alone formulas that are not part of a sentence and occupy their own line. For example:
RCA = \frac{(UL_A^2 + p \times UL_A \times UL_B)}{UL_p}

UL = A \times \sqrt{EDF \times VAR_{LGD} + LGD^2 \times VAR_{EDF}}\text{ Therefore:} should be converted to:

$$
RCA = \frac{(UL_A^2 + p \times UL_A \times UL_B)}{UL_p}
$$

$$
UL = A \times \sqrt{EDF \times VAR_{LGD} + LGD^2 \times VAR_{EDF}}\text{ Therefore:}
$$

4.If there are any tables in the image, the model must convert each row and column accurately into markdown format, ensuring that every row and column is consistent with the structure and content in the image. 
5.Do not include any other explanations, presentations, or formats that exceed the requirements. Focus only on the content of the image.
"""

                    },
                    # 使用 base64 编码传递图片
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_string}"
                        }
                    }
                ]
            }
        ],
        "max_tokens": 1000
    }
    
    # 发送请求
    response = requests.post("https://api.xi-ai.cn/v1/chat/completions", headers=headers, json=payload)
    response_json = response.json()
    
    # 检查响应内容并保存与图片文件对应的内容
    if 'choices' in response_json and len(response_json['choices']) > 0:
        response_content = response_json['choices'][0]['message']['content']
        # 将响应内容和图片文件名组合
        results[image_file] = {
            "image_file": image_file,
            "response_content": response_content
        }
    else:
        # 如果响应为空，记录错误信息
        results[image_file] = {
            "image_file": image_file,
            "response_content": "No valid response"
        }
    
    # 打印每个文件的响应
    print(f"Results for {image_file}: {json.dumps(results[image_file], indent=4)}")

<>:25: SyntaxWarning: invalid escape sequence '\s'
<>:25: SyntaxWarning: invalid escape sequence '\s'
C:\Users\18496\AppData\Local\Temp\ipykernel_8008\4017508631.py:25: SyntaxWarning: invalid escape sequence '\s'
  "text": """You will get an image with text and mathematical formulas. Your task is to extract content from images and convert it into a document format. The requirements are as follows:


Results for 17dn.png: {
    "image_file": "17dn.png",
    "response_content": "The value of Property 1 using the discounted cash flow method is $49,750,931, or $49,750,900 rounded, calculated as follows:\n\n|                          | Discount period | Discounted value*   |\n|--------------------------|-----------------|--------------------|\n| Year 1 NOI               | $2,775,840      | $2,588,196         |\n| Year 2 NOI               | $2,859,119      | $2,485,637         |\n| Year 3 NOI               | $2,944,889      | $2,387,135         |\n| Year 4 NOI               | $3,033,235      | $2,292,540         |\n| Year 5 NOI               | $3,124,232      | $2,201,693         |\n| Terminal Value**         | $53,632,650     | $37,795,731        |\n| Property #1 DCF value    |                 | $49,750,932        |\n\n* Discount rate = 7.25%\n\n** The terminal value = Year 6 NOI/terminal cap rate = $3,217,959/0.06 = $53,632,650"
}
Results for 18dn.png: {
    "image_file": "18dn.png",


In [61]:
results

{'17dn.png': {'image_file': '17dn.png',
  'response_content': 'The value of Property 1 using the discounted cash flow method is $49,750,931, or $49,750,900 rounded, calculated as follows:\n\n|                          | Discount period | Discounted value*   |\n|--------------------------|-----------------|--------------------|\n| Year 1 NOI               | $2,775,840      | $2,588,196         |\n| Year 2 NOI               | $2,859,119      | $2,485,637         |\n| Year 3 NOI               | $2,944,889      | $2,387,135         |\n| Year 4 NOI               | $3,033,235      | $2,292,540         |\n| Year 5 NOI               | $3,124,232      | $2,201,693         |\n| Terminal Value**         | $53,632,650     | $37,795,731        |\n| Property #1 DCF value    |                 | $49,750,932        |\n\n* Discount rate = 7.25%\n\n** The terminal value = Year 6 NOI/terminal cap rate = $3,217,959/0.06 = $53,632,650'},
 '18dn.png': {'image_file': '18dn.png',
  'response_content': 'The dir

In [71]:
# 假设 data 是一个列表，results 是字典，image_file 是文件名
for image_file, result in results.items():
    # 提取文件名中的数字部分作为题号
    file_number = ''.join(filter(str.isdigit, image_file))
    
    # 遍历 data 列表，找到匹配的题号并更新 'explanation' 字段
    for item in data:
        if item['question_number'] == file_number:
            # 更新 'explanation' 字段为 result['response_content']
            item['explanation'] = result['response_content']


In [72]:
# Convert the JSON data into a list of dictionaries with the required fields
data_list = []
for item in data:  # 直接遍历列表
    data_list.append({
        "ID": item.get("question_number", ""),  # 使用 question_number 作为 ID
        "question": item.get("question_text", ""),  # 访问 question_text
        "image": item.get("image", ""),  # 访问 image
        "options": item.get("options", {}),  # 访问 options
        "answer": item.get("answer", ""),
        "analysis": item.get("explanation", ""),  # 使用 explanation 作为 analysis
        "QA type": item.get("QA type", ""),
        "knowledge topics": item.get("knowledge topics", ""),
        "book label": item.get("book label", ""),
        "level of difficulty": item.get("level of difficulty", ""),
        "question type": item.get("question type", "")
    })


# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)
from IPython.display import display, Markdown
for index, row in df.iterrows():
  print(f"Question {index}")
    # Assuming 'ID' is a column in your DataFrame
  if 'ID' in df.columns:
    print(row['ID'])
  # Assuming 'analysis' is a column in your DataFrame
  if 'analysis' in df.columns:
    display(Markdown(row['analysis']))
  print("----------------------------------")

Question 0
1


Option 2, publicly traded REITs, best satisfy Caudill’s investment preferences.
REITs are equity investments that, in general, are income tax exempt at the corporate/trust
level, so there is no double income taxation. To qualify for the income tax exemption, REITs
are legally obligated to pay out a high percentage of income to their shareholders, and this
typically results in relatively high income for investors. Lastly, public REITs are generally
liquid as they are traded in stock exchanges.

----------------------------------
Question 1
2


Direct property ownership offers greater control over property level investment
decisions in comparison to the level of control exhibited by shareholders in REITs and REOCs.

----------------------------------
Question 2
3


...

----------------------------------
Question 50
51


Leverage needed to finance a management buyout is not readily available to firms
with limited history.

----------------------------------
Question 51
52


Statement 1 is the least likely to be valid.本题可以直接选出A选项，因为VC的现金流是
很难预测的，所以用现金流折现模型是最不切实际的。

----------------------------------


In [74]:
# 将 data 对象保存json文件
output_file = 'AlternativeInvestments.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

In [88]:
# Load the data from the file
with open(output_file, "r", encoding='utf-8') as f:
    data_change = json.load(f)


# Convert the JSON data into a list of dictionaries with the required fields
data_list_change = []
for item in data_change:  # 直接遍历列表
    data_list_change.append({
        "ID": item.get("question_number", ""),  # 使用 question_number 作为 ID
        "question": item.get("question_text", ""),  # 访问 question_text
        "image": item.get("image", ""),  # 访问 image
        "options": item.get("options", {}),  # 访问 options
        "answer": item.get("answer", ""),
        "analysis": item.get("explanation", ""),  # 使用 explanation 作为 analysis
        "QA type": item.get("QA type", ""),
        "knowledge topics": item.get("knowledge topics", ""),
        "book label": item.get("book label", ""),
        "level of difficulty": item.get("level of difficulty", ""),
        "question type": item.get("question type", "")
    })


# 将字典列表转换为 DataFrame
df = pd.DataFrame(data_list_change)
from IPython.display import display, Markdown

# 指定您想要打印的题目编号
specified_questions = [19]  # 将此列表替换为您想要的题目编号

# 确保 'ID' 列存在，并将其转换为整数类型
if 'ID' in df.columns:
    # 如果 'ID' 列中的值可能为空字符串，先替换为空值，然后再转换类型
    df['ID'] = df['ID'].replace('', pd.NA)
    df['ID'] = df['ID'].astype('Int64')  # 使用可为空的整数类型
else:
    print("DataFrame 中不存在 'ID' 列，请检查数据。")

# 筛选出指定的题目
df_filtered = df[df['ID'].isin(specified_questions)]

# 遍历筛选后的 DataFrame，打印指定题目的解析内容
for index, row in df_filtered.iterrows():
    print(f"Question {row['ID']}")
    # 如果 DataFrame 中有 'analysis' 列
    if 'analysis' in df.columns:
        display(Markdown(row['analysis']))
    print("----------------------------------")

Question 19


The estimate of the value of Property #1 using the sales comparison approach is:

|                  | Unadjusted psf | Adjusted psf               |
|------------------|----------------|----------------------------|
| Sales Comp 1     | $415           | $394 (= $415 \times 0.95)$ |
| Sales Comp 2     | $395           | $462 (= $395 \times 1.17)$ |
| Sales Comp 3     | $400           | $480 (= $400 \times 1.20)$ |
| **Average**      | $403           | $445                       |

Estimated Value of Property #1 = $44,500,000 = $445 psf \times 100,000 sf$

----------------------------------
